In [21]:
from _keys import db_user, db_password, db_name, db_host, db_port
import psycopg2
import pandas as pd

In [22]:
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
cur = conn.cursor()

In [23]:
sql = """
select t1.author, subreddit, submission, to_timestamp(time_created)::date as post_created, to_timestamp(account_created)::date as account_created, t1.is_bot from
	(select *
	from (
		select author, title as submission, created_utm as time_created, subreddit, 'post' as submission_type, 'True' as is_bot
		from sus_user_posts
		where title !='This account is banned and is temporarily preserved for purposes of transparency.') as posts_aggregate
	union
	select *
	from (
		select author, title as submission, created_utm as time_created, subreddit, 'post' as submission_type, 'False' as is_bot
		from norm_user_posts) as norm_posts_aggregate
	where subreddit in ( 
		select subreddit
		from sus_user_posts
		inner join subreddit_info on sus_user_posts.subreddit = subreddit_info.subreddit_name
		where over18 = 'False'and subreddit not in ('u_reddit', 'funny', 'gifs', 'aww', 'AnimalsBeingBros', 'corgi', 'gif', 'cats', 'memes', 'pics', 'dogpictures', 'puppies', 'UpliftingNews', 'celebrities')
		group by subreddit
		order by count(*) desc
		limit 30)) 
	as t1
inner join (
	select *
	from (
		select sus_user_name as author, account_created as account_created, 'True' as is_bot
		from sus_users
	) as sus_users
	union
	select  *
	from (
		select norm_user_name as author, account_created as account_created, 'False' as is_bot
		from norm_users
	) as norm_users
	)as t2 on t1.author = t2.author
limit 2000
"""

In [24]:
df = pd.read_sql(sql, conn)
df.to_csv('sample_data.csv', index=False)